# DX 704 Week 9 Project

This week's project will build an email spam classifier based on the Enron email data set.
You will perform your own feature extraction, and use naive Bayes to estimate the probability that a particular email is spam or not.
Finally, you will review the tradeoffs from different thresholds for automatically sending emails to the junk folder.

The full project description and a template notebook are available on GitHub: [Project 9 Materials](https://github.com/bu-cds-dx704/dx704-project-09).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [3]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import pandas as pd

In [5]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

Message ID                                            Subject  \
0               0                       christmas tree farm pictures   
1               1                           vastar resources , inc .   
2               2                       calpine daily gas nomination   
3               3                                         re : issue   
4               4                          meter 7268 nov allocation   
...           ...                                                ...   
33711       33711  = ? iso - 8859 - 1 ? q ? good _ news _ c = eda...   
33712       33712  all prescript medicines are on special . to be...   
33713       33713              the next generation online pharmacy .   
33714       33714                     bloow in 5 - 10 times the time   
33715       33715                   dear sir , i am interested in it   

                                                 Message Spam/Ham        Date  
0                                                    NaN      ham  1999-12-10  
1      gary , production from the high island larger ...      ham  1999-12-13  
2                 - calpine daily gas nomination 1 . doc      ham  1999-12-14  
3      fyi - see note below - already done .\nstella\...      ham  1999-12-14  
4      fyi .\n- - - - - - - - - - - - - - - - - - - -...      ham  1999-12-14  
...                                                  ...      ...         ...  
33711  hello , welcome to gigapharm onlinne shop .\np...     spam  2005-07-29  
33712  i got it earlier than expected and it was wrap...     spam  2005-07-29  
33713  are you ready to rock on ? let the man in you ...     spam  2005-07-30  
33714  learn how to last 5 - 10 times longer in\nbed ...     spam  2005-07-30  
33715  hi : )\ndo you need some softwares ? i can giv...     spam  2005-07-31  

[33716 rows x 5 columns]

In [6]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Design a Feature Extractor

Design a feature extractor for this data set and write out two files of features based on the text.
Don't forget that both the Subject and Message columns are relevant sources of text data.
For each email, you should count the number of repetitions of each feature present.
The auto-grader will assume that you are using a multinomial distribution in the following problems.

In [13]:
# Design Feature extractor for the subject and message fields
from sklearn.feature_extraction.text import CountVectorizer
import json

# Combine Subject and Message into a single text field
# Subject lines are often more indicative, so we can give them more weight by including them twice
enron_spam_data['combined_text'] = (
    enron_spam_data['Subject'].fillna('') + ' ' + enron_spam_data['Message'].fillna('')
)

# Configure CountVectorizer with good spam detection parameters
vectorizer = CountVectorizer(
    lowercase=True,           # Normalize case
    ngram_range=(1, 1),       # Use unigrams and bigrams (captures phrases like "click here")
    stop_words=None           # Keep stop words - they can be informative for spam
)

# Fit and transform on the combined text
feature_matrix = vectorizer.fit_transform(enron_spam_data['combined_text'])
feature_names = vectorizer.get_feature_names_out()

print(f"Extracted {len(feature_names)} features")
print(f"Feature matrix shape: {feature_matrix.shape}")
print(f"Sample features: {list(feature_names[:20])}")

Extracted 158285 features
Feature matrix shape: (33716, 158285)
Sample features: ['00', '000', '0000', '000000', '00000000', '0000000000', '0000000000001110', '000000000002858', '000000000003619', '000000000003991', '000000000003997', '000000000005168', '000000000005409', '000000000005411', '000000000005412', '000000000005413', '000000000005820', '000000000006238', '000000000006452', '000000000007399']


Assign a row to the test data set if `Message ID % 30 == 0` and assign it to the training data set otherwise.
Write two files, "train-features.tsv" and "test-features.tsv" with two columns, Message ID and features_json.
The features_json column should contain a JSON dictionary where the keys are your feature names and the values are integer feature values.
This will give us a sparse feature representation.


In [16]:
# Split data: test if Message ID % 30 == 0, otherwise train
train_mask = enron_spam_data['Message ID'] % 30 != 0
test_mask = enron_spam_data['Message ID'] % 30 == 0

train_data = enron_spam_data[train_mask].copy()
test_data = enron_spam_data[test_mask].copy()

print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# Convert feature matrix to sparse JSON format
def create_features_json(feature_matrix, feature_names):
    """Convert scipy sparse matrix to list of JSON dictionaries with non-zero counts"""
    features_list = []
    
    for i in range(feature_matrix.shape[0]):
        # Get non-zero features for this row
        row = feature_matrix[i]
        non_zero_indices = row.nonzero()[1]
        
        # Create dictionary with feature names and counts
        feature_dict = {
            feature_names[idx]: int(row[0, idx])
            for idx in non_zero_indices
        }
        
        features_list.append(json.dumps(feature_dict))
    
    return features_list

# Get feature indices for train and test splits
train_indices = train_data.index
test_indices = test_data.index

# Create features JSON for train and test
train_features_json = create_features_json(feature_matrix[train_indices], feature_names)
test_features_json = create_features_json(feature_matrix[test_indices], feature_names)

# Create DataFrames with Message ID and features_json
train_output = pd.DataFrame({
    'Message ID': train_data['Message ID'].values,
    'features_json': train_features_json
})

test_output = pd.DataFrame({
    'Message ID': test_data['Message ID'].values,
    'features_json': test_features_json
})

# Save to TSV files
train_output.to_csv('train-features.tsv', sep='\t', index=False)
test_output.to_csv('test-features.tsv', sep='\t', index=False)


Training samples: 32592
Test samples: 1124


In [15]:
print("\nSample train features:")
print(train_output.head(3))
print(f"\nSample feature JSON: {train_features_json[0][:200]}...")

print("\nSample test features:")
print(test_output.head(3))
print(f"\nSample feature JSON: {test_features_json[0][:200]}...")


Sample train features:
   Message ID                                      features_json
0           1  {"vastar": 6, "resources": 4, "inc": 4, "gary"...
1           2  {"nomination": 2, "gas": 2, "calpine": 2, "dai...
2           3  {"from": 3, "on": 4, "at": 1, "10": 3, "for": ...

Sample feature JSON: {"vastar": 6, "resources": 4, "inc": 4, "gary": 2, "production": 3, "from": 3, "the": 8, "high": 2, "island": 2, "larger": 2, "block": 2, "commenced": 1, "on": 7, "saturday": 1, "at": 3, "00": 2, "abo...

Sample test features:
   Message ID                                      features_json
0           0  {"christmas": 1, "tree": 1, "farm": 1, "pictur...
1          30  {"christmas": 1, "production": 3, "from": 1, "...
2          60  {"the": 1, "of": 1, "by": 1, "please": 1, "cal...

Sample feature JSON: {"christmas": 1, "tree": 1, "farm": 1, "pictures": 1}...


Submit "train-features.tsv" and "test-features.tsv" in Gradescope.

Hint: these features will be graded based on the test accuracy of a logistic regression based on the training features.
This is to make sure that your feature set is not degenerate; you do not need to compute this regression yourself.
You can separately assess your feature quality based on your results in part 6.

## Part 3: Compute Conditional Probabilities

Based on your training data, compute appropriate conditional probabilities for use with naïve Bayes.
Use of additive smoothing with $\alpha=1$ to avoid zeros.


In [ ]:
# YOUR CHANGES HERE

...

Save the conditional probabilities in a file "feature-probabilities.tsv" with columns feature, ham_probability and spam_probability.

In [ ]:
# YOUR CHANGES HERE

...

Submit "feature-probabilities.tsv" in Gradescope.

## Part 4: Implement a Naïve Bayes Classifier

Implement a naïve Bayes classifier based on your previous feature probabilities.

In [ ]:
# YOUR CHANGES HERE

...

Save your prediction probabilities to "train-predictions.tsv" with columns Message ID, ham and spam.

In [ ]:
# YOUR CHANGES HERE

...

Submit "train-predictions.tsv" in Gradescope.

## Part 5: Predict Spam Probability for Test Data

Use your previous classifier to predict spam probability for the test data.

In [ ]:
# YOUR CHANGES HERE

...

Save your prediction probabilities in "test-predictions.tsv" with the same columns as "train-predictions.tsv".

In [ ]:
# YOUR CHANGES HERE

...

Submit "test-predictions.tsv" in Gradescope.

## Part 6: Construct ROC Curve

For every probability threshold from 0.01 to .99 in increments of 0.01, compute the false and true positive rates from the test data using the spam class for positives.
That is, if the predicted spam probability is greater than or equal to the threshold, predict spam.

In [ ]:
# YOUR CHANGES HERE

...

Save this data in a file "roc.tsv" with columns threshold, false_positive_rate and true_positive rate.

In [ ]:
# YOUR CHANGES HERE

...

Submit "roc.tsv" in Gradescope.

## Part 7: Signup for Gemini API Key

Create a free Gemini API key at https://aistudio.google.com/app/api-keys.
You will need to do this with a personal Google account - it will not work with your BU Google account.
This will not incur any charges unless you configure billing information for the key.

You will be asked to start a Gemini free trial for week 11.
This will not incur any charges unless you exceed expected usage by an order of magnitude.


No submission needed.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.